In [1]:
import os
# print(os.getcwd())
os.chdir('D:\OneDrive - zju.edu.cn\Documents\Git\Machine-learning\Machine learning kaggle\IMDB')
print(os.getcwd())

d:\OneDrive - zju.edu.cn\Documents\Git\Machine-learning\Machine learning kaggle\IMDB


In [2]:
import pandas as pd

train_data = pd.read_csv('datasets/labeledTrainData.tsv',delimiter='\t')
test_data = pd.read_csv('datasets/testData.tsv',delimiter='\t')
print(train_data.head())
print(test_data.head())


       id  sentiment                                             review
0  5814_8          1  With all this stuff going down at the moment w...
1  2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2  7759_3          0  The film starts with a manager (Nicholas Bell)...
3  3630_4          0  It must be assumed that those who praised this...
4  9495_8          1  Superbly trashy and wondrously unpretentious 8...
         id                                             review
0  12311_10  Naturally in a film who's main themes are of m...
1    8348_2  This movie is a disaster within a disaster fil...
2    5828_4  All in all, this is a movie for kids. We saw i...
3    7186_2  Afraid of the Dark left me with the impression...
4   12128_7  A very accurate depiction of small time mob li...


In [3]:
# review = train_data.loc[0,'review']
# print('review:\n',review)

# stop_words = set(stopwords.words('english'))
# def review_to_text(review,remove_stopwords):
#     raw_text = BeautifulSoup(review,'html').get_text()
#     letters = re.sub('[^a-zA-Z]',' ',raw_text)
#     words = letters.lower().split()
#     print('words:\n',words)
#     if remove_stopwords: 
#         words = [w for w in words if w not in stop_words]
#     print('words:\n',words)
#     return words

# x_train = []
# x_train.append(' '.join(review_to_text(review,True)))
# print('x_train:\n',x_train)

In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
def review_to_text(review,remove_stopwords):
    raw_text = BeautifulSoup(review,'html').get_text()
    letters = re.sub('[^a-zA-Z]',' ',raw_text)
    words = letters.lower().split()
    if remove_stopwords: 
        words = [w for w in words if w not in stop_words]
    return words

x_train = []
for review in train_data['review']:
    x_train.append(' '.join(review_to_text(review,True)))
x_test = []
for review in test_data['review']:
    x_test.append(' '.join(review_to_text(review,True)))

y_train = train_data['sentiment']
# print(x_train[:10])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pip_count = Pipeline([('count_vec',CountVectorizer(analyzer='word')),('mnb',MultinomialNB())])
pip_tfidf = Pipeline([('tfidf_vec',TfidfVectorizer(analyzer='word')),('mnb',MultinomialNB())])

params_count = {'count_vec__binary':[True,False],'count_vec__ngram_range':[(1,1),(1,2)],'mnb__alpha':[0.1,1,10.0]}
params_tfidf = {'tfidf_vec__binary':[True,False],'tfidf_vec__ngram_range':[(1,1),(1,2)],'mnb__alpha':[0.1,1,10.0]}

gs_count = GridSearchCV(pip_count,params_count,cv=4,n_jobs=-1,verbose=1)
gs_count.fit(x_train,y_train)
print('gs_count.best_score_:',gs_count.best_score_)
print('gs_count.best_params_:',gs_count.best_params_)
count_y_predict = gs_count.predict(x_test)

gs_tfidf = GridSearchCV(pip_tfidf,params_tfidf,cv=4,n_jobs=-1,verbose=1)
gs_tfidf.fit(x_train,y_train)
print('gs_tfidf.best_score_:',gs_tfidf.best_score_)
print('gs_tfidf.best_params_:',gs_tfidf.best_params_)
tfidf_y_predict = gs_tfidf.predict(x_test)


Fitting 4 folds for each of 12 candidates, totalling 48 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.7min finished
gs_count.best_score_: 0.88216
gs_count.best_params_: {'count_vec__binary': True, 'count_vec__ngram_range': (1, 2), 'mnb__alpha': 1}
Fitting 4 folds for each of 12 candidates, totalling 48 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.3min finished
gs_tfidf.best_score_: 0.88712
gs_tfidf.best_params_: {'mnb__alpha': 0.1, 'tfidf_vec__binary': True, 'tfidf_vec__ngram_range': (1, 2)}


In [6]:
submission_count = pd.DataFrame({'id':test_data['id'],'sentiment':count_y_predict})
submission_tfidf = pd.DataFrame({'id':test_data['id'],'sentiment':tfidf_y_predict})

submission_count.to_csv('datasets/submission_count.csv',index=False)
submission_tfidf.to_csv('datasets/submission_tfidf.csv',index=False)

In [7]:
unlabeled_data = pd.read_csv('datasets/unlabeledTrainData.tsv',delimiter='\t',quoting=3)        #quoting=3   如实读取引号内容
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = []

def new2sentences(review,sentences):
    news_text = BeautifulSoup(review).get_text()
    # print('news_text:\n',news_text)
    raw_sentences = tokenizer.tokenize(news_text)
    # print('raw_sentences:\n',raw_sentences)
    for raw_sentence in raw_sentences:
        sentences.append(re.sub('[^a-zA-Z]',' ',raw_sentence.lower().strip()).split())
    return

for review in unlabeled_data['review']:
    # print('review:\n',review)
    new2sentences(review,sentences)

# from sklearn.datasets import fetch_20newsgroups
# news = fetch_20newsgroups(subset='all')
# x,y = news.data,news.target
# for review in x:
#     new2sentences(review,sentences)

print('sentences:\n',sentences[0:10])
print(len(sentences))


corpora:
 [['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends'], ['maybe', 'they', 'were', 'sitting', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'said', 'hey', 'let', 's', 'pool', 'our', 'money', 'together', 'and', 'make', 'a', 'really', 'bad', 'movie', 'or', 'something', 'like', 'that'], ['what', 'ever', 'they', 'said', 'they', 'still', 'ended', 'up', 'making', 'a', 'really', 'bad', 'movie', 'dull', 'story', 'bad', 'script', 'lame', 'acting', 'poor', 'cinematography', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', 'etc'], ['all', 'corners', 'were', 'cut', 'except', 'the', 'one', 'that', 'would', 'have', 'prevented', 'this', 'film', 's', 'release'], ['life', 's', 'like', 'that'], ['i', 'saw', 'this', 'film', 'about', 'years', 'ago', 'and', 'remember', 'it', 'as', 'being', 'particularly', 'nasty'], ['i', 'believe', 'it', 'is', 'based', 'on', 'a', 'true', 'incident', 'a', 'young', 'man', 'breaks', 'into', 'a', 

In [9]:
from gensim.models import word2vec
num_features = 300
min_word_count = 20
num_workers = 4
context = 10
downsampling = 1e-3

model = word2vec.Word2Vec(sentences,workers=num_workers, \
    size=num_features, min_count=min_word_count, \
    window=context, sample=downsampling)
model.init_sims(replace=True)
model_name = 'datasets/300feature_20minwords_10context'
print(model.most_similar('email'))
model.save(model_name)


[('yahoo', 0.7360875606536865), ('mail', 0.6054712533950806), ('http', 0.5840326547622681), ('trust', 0.5825950503349304), ('com', 0.5734444856643677), ('woe', 0.5710731148719788), ('www', 0.5508212447166443), ('blogspot', 0.5481081008911133), ('atheist', 0.5228750109672546), ('grabbed', 0.522038459777832)]


In [10]:
from gensim.models import Word2Vec
model = Word2Vec.load(model_name)
print(model.most_similar('man'))

[('woman', 0.6411173343658447), ('lad', 0.6086446642875671), ('lady', 0.5957801342010498), ('person', 0.5326814651489258), ('men', 0.5281357169151306), ('chap', 0.5262560844421387), ('boy', 0.523442268371582), ('soldier', 0.5227000713348389), ('guy', 0.5177127122879028), ('monk', 0.5101040601730347)]


In [11]:
import numpy as np
def make_feature_vec(words,model,num_features):
    feature_vec = np.zeros(num_features,dtype='float32')
    n_words = 0.0
    index2words_set = set(model.wv.index2word)
    for word in words:
        if word in index2words_set:
            n_words += 1
            feature_vec = feature_vec + model[word]
    feature_vec = np.divide(feature_vec,n_words)
    return feature_vec

# 将每一条影评转换为基于词向量的特征向量
def get_average_feature_vec(reviews,model,num_features):
    counter = 0
    review_feature_vecs = np.zeros([len(reviews),num_features],dtype='float32')
    for review in reviews:
        review_feature_vecs[counter] = make_feature_vec(review,model,num_features)
        counter += 1
    return review_feature_vecs

clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(review_to_text(review,remove_stopwords=True))
train_data_vecs = get_average_feature_vec(clean_train_reviews,model,num_features)
clean_test_reviews = []
for review in test_data['review']:
    clean_test_reviews.append(review_to_text(review,remove_stopwords=True))
test_data_vecs = get_average_feature_vec(clean_test_reviews,model,num_features)



In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
# params_gbc = {'n_estimators':[10,100,500],'learning_rate':[0.01,0.1,1.0],'max_depth':[2,3,4]}
params_gbc = {'n_estimators':[500],'learning_rate':[0.1],'max_depth':[4]}
gs = GridSearchCV(gbc,params_gbc,cv=4,n_jobs=-1,verbose=1)
gs.fit(train_data_vecs,y_train)
print('gs.best_score_:',gs.best_score_)
print('gs.best_params_:',gs.best_params_)
v2c_y_predict=gs.predict(test_data_vecs)
submission_v2c = pd.DataFrame({'id':test_data['id'],'sentiment':v2c_y_predict})
submission_v2c.to_csv('datasets/submission_v2c.csv',index=False)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 18.1min finished
gs.best_score_: 0.85808
gs.best_params_: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500}
